In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/partial_clustering_random_percentage.zip" "/content"

In [ ]:
!unzip "/content/partial_clustering_random_percentage.zip"

Archive:  /content/partial_clustering_random_percentage.zip
   creating: partial_clustering_random_percentage/
  inflating: partial_clustering_random_percentage/vgg_pruningmethod.py  
  inflating: partial_clustering_random_percentage/partial_clustering.py  
  inflating: partial_clustering_random_percentage/VGG16.py  
  inflating: partial_clustering_random_percentage/vgg_network_prune.py  
  inflating: partial_clustering_random_percentage/epoch_295.pth  
   creating: partial_clustering_random_percentage/__pycache__/
  inflating: partial_clustering_random_percentage/__pycache__/vgg_pruningmethod.cpython-36.pyc  
  inflating: partial_clustering_random_percentage/__pycache__/vgg_network_prune.cpython-36.pyc  
  inflating: partial_clustering_random_percentage/__pycache__/VGG16.cpython-36.pyc  
   creating: partial_clustering_random_percentage/data/
  inflating: partial_clustering_random_percentage/data/cifar-10-python.tar.gz  
   creating: partial_clustering_random_percentage/checkpoint/
  

In [ ]:
!cp "/content/partial_clustering_random_percentage/epoch_295.pth" "/content"

In [ ]:
!cp "/content/partial_clustering_random_percentage/checkpoint/ckpt.t7" "/content/checkpoint"

vgg_network_prune

In [ ]:

import torch as th
import torch.nn as nn
import torch.nn.init as init
from numpy.linalg import norm
#import pandas as pd
import numpy as np
import logging
import csv 
from time import localtime, strftime
import os 
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift, estimate_bandwidth
import scipy.cluster.hierarchy as hcluster
import scipy.cluster.hierarchy as hac
import scipy.cluster.hierarchy as fclusterdata
import time
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from sklearn.metrics import pairwise_distances
import math

seed = 1787
#random.seed(seed)
#import os
#os.environ['PYTHONHASHSEED'] = str(seed)

#th.manual_seed(seed)
#th.cuda.manual_seed(seed)
#th.cuda.manual_seed_all(seed)
#th.backends.cudnn.deterministic = True


class Network():

    def weight_init(self, m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            if self.a_type == 'relu':
                init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'leaky_relu':
                init.kaiming_normal_(m.weight.data, nonlinearity=self.a_type)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'tanh':
                g = init.calculate_gain(self.a_type)
                init.xavier_uniform_(m.weight.data, gain=g)
                init.constant_(m.bias.data, 0)
            elif self.a_type == 'sigmoid':
                g = init.calculate_gain(self.a_type)
                init.xavier_uniform_(m.weight.data, gain=g)
                init.constant_(m.bias.data, 0)
            else:
                raise
                return NotImplemented

    
    def create_folders(self,total_convs):

      main_dir=strftime('/content/drive/MyDrive/BTP/sem7\\Results\\%b%d_%H_%M_%S%p', localtime() )+"_vgg"

      # temp=strftime('%b%d_%H:%M:%S%p', localtime() )+"_vgg"
      # temp = "\\Sep27_21:"
      # print(temp)

      import os
      current_dir =  os.path.abspath(os.path.dirname("/content/drive/MyDrive/BTP/sem7"))
      par_dir = os.path.abspath(current_dir + "/../")
      print(par_dir)
      par_dir=os.path.dirname(os.path.realpath("/content"))
      print(par_dir)
      parent_dir=par_dir+main_dir
 
      for i in range(total_convs):
        path1=os.path.join(main_dir, "conv"+str(i+1))
        print(path1)
        os.makedirs(path1)
      return parent_dir

    def get_writerow(self,k):

      s='wr.writerow(['

      for i in range(k):

          s=s+'d['+str(i)+']'

          if(i<k-1):
             s=s+','
          else:
             s=s+'])'

      return s

    def get_logger(self,file_path):

        logger = logging.getLogger('gal')
        log_format = '%(asctime)s | %(message)s'
        formatter = logging.Formatter(log_format, datefmt='%m/%d %I:%M:%S %p')
        file_handler = logging.FileHandler(file_path)
        file_handler.setFormatter(formatter)
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)

        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)
        logger.setLevel(logging.INFO)

        return logger

    def cluster_weights_agglo(self,weight, threshold,prunes,folder_name,conv_layer,first=False, average=True):
        #t0 = time.time()
        #print(type(weight))
        #print('1...',weight.shape)
        weight = weight.T
        #print('1...',weight.shape, type(weight))
        weight = normalize(weight, norm='l2', axis=1)
        #print('2...',weight.shape)
        threshold =  1.0-threshold   # Conversion to distance measure
        #clusters = hcluster.fclusterdata(weight, threshold, criterion="distance", metric='cosine', depth=1, method='centroid')
        z = hac.linkage(weight, metric='cosine', method='complete')

        #print('z.......',len(z))
        labels = hac.fcluster(z, threshold, criterion="distance")
        #print('lables...',labels)
       
        #print('sorted lables...',len(labels),np.sort(labels))

        labels_unique = np.unique(labels)
        #print('labels_unique...',labels_unique.shape,labels_unique)
        n_clusters_ = len(labels_unique)

        a=np.array(labels)
        #print('a....',a)
        sort_idx = np.argsort(a)
        #print('sorted idx...',len(sort_idx), sort_idx)
        a_sorted1 = a[sort_idx]
        #print('a_sorted...',len(a_sorted1), a_sorted1, type(a_sorted1))


        if(first==False): #ONLY 1 from each cluster

            unq_first = np.concatenate(([True], a_sorted1[1:] != a_sorted1[:-1]))
            #print('unq_first...',len(unq_first), unq_first)

            unq_items = a_sorted1[unq_first]
            unq_count = np.diff(np.nonzero(unq_first)[0])
            unq_idx = np.split(sort_idx, np.cumsum(unq_count))
            first_ele = [unq_idx[idx][-1] for idx in range(len(unq_idx))]
        else: # % of filters from each cluster
            a_sorted= np.sort(labels)
            val2= sort_idx
            temp=[] 
            filters=[]      
            for i in range(len(a_sorted)):
			   
                val=a_sorted[i]
		
                if (i== len(a_sorted)-1 ):
                      #print('True', val2[i])
                      temp.append(val2[i])
                      filters.append(temp)
                      temp=[]
				
                elif(i!= len(a_sorted)-1 and (val != a_sorted[i+1])):
                      #print('True', val2[i])
                      temp.append(val2[i])
                      filters.append(temp)
                      temp=[]
                else:
                      #print('False')
                      temp.append(val2[i])
            #print(filters)

            #sort each cluster based on each weighs L1/L2 norm and plot histogram
            l1_norms=[]
            for i in range(len(filters)):
               temp=[]
               for f in filters[i]:
                 #print(i,'...',weight[f].shape,'....',norm(weight[f],2), norm(weight[f],1))
                 #print(weight[f])
                 temp.append(norm(weight[f],1)) #L1-norm
               l1_norms.append(np.around(temp,3))  

            #____________SORTING FILTERS BASED ON L1 NORM________
            '''sorted_filters=[]
            for i in range(len(filters)):
                 sorted_filters.append( [x for _,x in sorted(zip(l1_norms[i],filters[i]))] )'''


            # for i in range(len(l1_norms)):
            #      plt.hist(l1_norms[i], weights=np.ones(len(l1_norms[i])) / len(l1_norms[i]))
            #      plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
            #      plt.savefig(folder_name+"/c"+str(i+1)+".png")
            #      plt.clf()
            #      plt.cla()
            #      plt.close()



            #filters=sorted_filters #_____COMMENT OUT FOR random pruning___
            ele=[]
            for j in range(len(filters)):

                if(len(filters[j])==1):
                  cut=0
                else:
                  cut= math.ceil(len(filters[j])/4) #prune 25% only

                retain_filters= filters[j][cut:]
                ele.extend(retain_filters)
                #print('total=',len(filters[j]),' remaining=',len(retain_filters),'  index=',cut)
            #print(ele)
            first_ele=ele

        

        #print('first ele.....',len(first_ele))
        return n_clusters_, first_ele


vgg_pruningmethod

In [ ]:
import torch as th
import torch.nn as nn 
import numpy as np
import os

#seed = 1787
#np.random.seed(seed)

filters_selected=[]
#filters_remaining=[]
no_of_dimensions=-1

#prune_percentage=[0.02]*2+[0.04]*2+[0.05]*3+[0.1]*6
class PruningMethod():

   
    def prune_filters(self,threshold,prunes,folder_name):
      conv_layer=0
      total=None #-------
      first_ele= None #-------
      #nb_remanining_filters = []
      print(prunes)
  
      for layer_name, layer in self.named_modules():
        #print(layer_name)

        if isinstance(layer, nn.Conv2d):

            #CREATE PRUNE STEP FOLDER
            path1=os.path.join(folder_name+"conv"+str(conv_layer+1)+"/prune"+str(prunes))
            os.makedirs(path1)

            #print(layer_name)
            weight = layer.weight.data.cpu().numpy()
            bias = layer.bias.data.cpu().numpy()
            #print('1..',weight.shape)
            if first_ele is not None:
                weight_layers_rearranged = np.transpose(weight, (1, 0, 2, 3))
                weight_layers_rearranged_pruned = weight_layers_rearranged[first_ele]
                weight_layers_rearranged_pruned = np.transpose(weight_layers_rearranged_pruned, (1, 0, 2, 3))
            else:
                weight_layers_rearranged_pruned = weight
            #print('2a...',weight_layers_rearranged_pruned.shape)
            weight_layers_rearranged = np.reshape(weight_layers_rearranged_pruned, [weight_layers_rearranged_pruned.shape[0], -1])
            #print('3a.....',weight_layers_rearranged.shape)
            if(prunes>20): #if(prunes!=0):
               #print('NORMAL  calling.....')
               n_clusters_,first_ele = self.cluster_weights_agglo(weight_layers_rearranged.T, threshold,prunes,path1,conv_layer) 
            else:
               #print('calling.....')
               n_clusters_,first_ele = self.cluster_weights_agglo(weight_layers_rearranged.T, threshold,prunes,path1,conv_layer, first=True) #50% pruning


            conv_layer+=1



            first_ele = sorted(first_ele)

            weight_pruned = weight_layers_rearranged[first_ele]
            #print('4..',weight_pruned.shape)
            bias_pruned = bias[first_ele]


            weight_pruned = np.reshape(weight_pruned, [len(first_ele), weight_layers_rearranged_pruned.shape[1],weight_layers_rearranged_pruned.shape[2],weight_layers_rearranged_pruned.shape[3]])
            #print('5..',weight_pruned.shape)

            params_1 = np.shape(weight_pruned)
            layer.out_channels = params_1[0]
            layer.in_channels = params_1[1]

            weight_tensor = th.from_numpy(weight_pruned)
            bias_tensor = th.from_numpy(bias_pruned)
            layer.weight = th.nn.Parameter(weight_tensor)
            layer.bias = th.nn.Parameter(bias_tensor)

           
            #ii+=1
            #rr+=1

        if isinstance(layer, nn.BatchNorm2d) and first_ele is not None:
            bnorm_weight = layer.weight.data.cpu().numpy()
            bnorm_weight = bnorm_weight[first_ele]
            bnorm_bias = layer.bias.data.cpu().numpy()
            bnorm_bias = bnorm_bias[first_ele]

            bnorm_tensor = th.from_numpy(bnorm_weight)
            bias_tensor = th.from_numpy(bnorm_bias)
            layer.weight = th.nn.Parameter(bnorm_tensor)
            layer.bias = th.nn.Parameter(bias_tensor)

            layer.num_features = int(np.shape(bnorm_weight)[0])
            bnorm_rm = layer.running_mean.cpu().numpy()
            bnorm_rm = bnorm_rm[first_ele]
            bnorm_rv = layer.running_var.cpu().numpy()
            bnorm_rv = bnorm_rv[first_ele]
            running_mean = th.from_numpy(bnorm_rm)
            layer.running_mean = running_mean
            running_var = th.from_numpy(bnorm_rv)
            layer.running_var = running_var
            #rr+=1

        if isinstance(layer, nn.Linear):
            weight_linear = layer.weight.data.cpu().numpy()
            #print('1...',weight_linear.shape)
            weight_linear_rearranged = np.transpose(weight_linear, (1, 0))
            weight_linear_rearranged_pruned = weight_linear_rearranged[first_ele]
            weight_linear_rearranged_pruned = np.transpose(weight_linear_rearranged_pruned, (1, 0))
            #print('2....',weight_linear_rearranged_pruned.shape)

            layer.in_features = int(np.shape(weight_linear_rearranged_pruned)[1])
            linear_tensor = th.from_numpy(weight_linear_rearranged_pruned)
            layer.weight = th.nn.Parameter(linear_tensor)
            break

           
            

      
    def get_indices_topk(self,layer_bounds,i,prune_limit,prune_percentage):

      #global prune_percentage
      indices=int(len(layer_bounds)*prune_percentage[i])+1 #1
      p=len(layer_bounds) #3
      if (p-indices)<prune_limit: #3-1<3
         remaining=p-prune_limit
         indices=remaining
      k=sorted(range(len(layer_bounds)), key=lambda j: layer_bounds[j])[:indices]
      #print('indidces',k, 'len ',len(layer_bounds))
      return k

    def get_indices_bottomk(self,layer_bounds,i,prune_limit):

      k=sorted(range(len(layer_bounds)), key=lambda j: layer_bounds[j])[-prune_limit:]
      #print('indidces',k, 'len ',len(layer_bounds))
      return k



vgg16

In [ ]:
import torch as th
import torch.nn as nn
# from vgg_network_prune import Network
# from  vgg_pruningmethod import PruningMethod
#from torchsummary import summary
import math
from collections import OrderedDict

class VGG16(nn.Module,Network,PruningMethod):
    def __init__(self, n_c, a_type):
        super(VGG16, self).__init__()

        self.a_type = a_type

        if a_type == 'relu':
            self.activation = nn.ReLU()
        elif a_type == 'tanh':
            self.activation = nn.Tanh()
        elif a_type == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif a_type == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        else:
            print('Not implemented')
            raise

        # First encoder
        self.layer1 = nn.Sequential(
                *([nn.Conv2d(3, 64, kernel_size=3, padding=1),
                   nn.BatchNorm2d(64),
                   self.activation]))
        self.layer2 = nn.Sequential(
                *([nn.Conv2d(64, 64, kernel_size=3, padding=1),
                   nn.BatchNorm2d(64),
                   self.activation]))
        # Second encoder
        self.layer3 = nn.Sequential(
                *([nn.Conv2d(64, 128, kernel_size=3, padding=1),
                   nn.BatchNorm2d(128),
                   self.activation]))
        self.layer4 = nn.Sequential(
                *([nn.Conv2d(128, 128, kernel_size=3, padding=1),
                   nn.BatchNorm2d(128),
                   self.activation]))

        # Third encoder
        self.layer5 = nn.Sequential(
                *([nn.Conv2d(128, 256, kernel_size=3, padding=1),
                   nn.BatchNorm2d(256),
                   self.activation]))
        self.layer6 = nn.Sequential(
                *([nn.Conv2d(256, 256, kernel_size=3, padding=1),
                   nn.BatchNorm2d(256),
                   self.activation]))
        self.layer7 = nn.Sequential(
                *([nn.Conv2d(256, 256, kernel_size=3, padding=1),
                   nn.BatchNorm2d(256),
                   self.activation]))

        # Fourth encoder
        self.layer8 = nn.Sequential(
                *([nn.Conv2d(256, 512, kernel_size=3, padding=1),
                   nn.BatchNorm2d(512),
                   self.activation]))
        self.layer9 = nn.Sequential(
                *([nn.Conv2d(512, 512, kernel_size=3, padding=1),
                   nn.BatchNorm2d(512),
                   self.activation]))
        self.layer10 = nn.Sequential(
                *([nn.Conv2d(512, 512, kernel_size=3, padding=1),
                   nn.BatchNorm2d(512),
                   self.activation]))

        # Fifth encoder
        self.layer11 = nn.Sequential(
                *([nn.Conv2d(512, 512, kernel_size=3, padding=1),
                   nn.BatchNorm2d(512),
                   self.activation]))
        self.layer12 = nn.Sequential(
                *([nn.Conv2d(512, 512, kernel_size=3, padding=1),
                   nn.BatchNorm2d(512),
                   self.activation]))
        self.layer13 = nn.Sequential(
                *([nn.Conv2d(512, 512, kernel_size=3, padding=1),
                   nn.BatchNorm2d(512),
                   self.activation]))

        # Classifier
        self.fc1 = nn.Sequential(*([
                nn.Linear(512, 512),
                nn.BatchNorm1d(512),
                self.activation]))

        self.classifier = nn.Sequential(
                *([nn.Linear(512, n_c),]))

        for m in self.modules():
            self.weight_init(m)

        self.pool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)


        self.layer_name_num={}
        self.pruned_filters={}
        self.remaining_filters={}

        self.remaining_filters_each_epoch=[]

       

    def forward(self, x):

        # Encoder 2
        layer1 = self.layer1(x)
        layer2 = self.layer2(layer1)
        pool1 = self.pool(layer2)

        # Encoder 2
        layer3 = self.layer3(pool1)
        layer4 = self.layer4(layer3)
        pool2 = self.pool(layer4)

        # Encoder 3
        layer5 = self.layer5(pool2)
        layer6 = self.layer6(layer5)
        layer7 = self.layer7(layer6)
        pool3 = self.pool(layer7)

        # Encoder 4
        layer8 = self.layer8(pool3)
        layer9 = self.layer9(layer8)
        layer10 = self.layer10(layer9)
        pool4 = self.pool(layer10)

        # Encoder 5
        layer11 = self.layer11(pool4)
        layer12 = self.layer12(layer11)
        
        layer13 = self.layer13(layer12)
        #pool5 = self.pool(layer13)
        #print(layer13.shape)

        #avgpool 
        avg_x=nn.AvgPool2d(2)(layer13)
        #print(avg_x.shape)

        # Classifier
        fc1 = self.fc1(avg_x.view(avg_x.size(0), -1))
        #print(fc1.shape)
        
        classifier = self.classifier(fc1)
        return classifier


'''device = th.device("cuda" if th.cuda.is_available() else "cpu") # PyTorch v0.4.0
model_tupry = VGG16(10,'relu',100).to(device)

a=model_tupry(th.randn(1, 3, 32, 32).cuda())
for i in range(15):
	print('layer ',str(i),' ',a[i].shape)
#for name, layer_module in model_tupry.named_modules():
#  if(isinstance(layer_module, th.nn.Conv2d)):
#    a.append(layer_module.shape)
#---------intialize layer numbers and names dictionary------
model_tupry.intialize_layer_name_num(model_tupry)

#---------intialize pruned and remaning filters-----------
model_tupry.filters_in_each_layer(model_tupry)
 
#--------calculate remaining filters in each epoch--------
model_tupry.remaining_filters_per_epoch(model=model_tupry,initial=True)
model_tupry.calculate_total_flops(model_tupry)
import sys
sys.exit()
for layer_name, layer_module in model_tupry.named_modules():
           if(isinstance(layer_module, th.nn.Conv2d) or  isinstance(layer_module,th.nn.Linear)):
              print(layer_name,'-------',layer_module,'---------',layer_module.weight.size()[0])'''



'device = th.device("cuda" if th.cuda.is_available() else "cpu") # PyTorch v0.4.0\nmodel_tupry = VGG16(10,\'relu\',100).to(device)\n\na=model_tupry(th.randn(1, 3, 32, 32).cuda())\nfor i in range(15):\n\tprint(\'layer \',str(i),\' \',a[i].shape)\n#for name, layer_module in model_tupry.named_modules():\n#  if(isinstance(layer_module, th.nn.Conv2d)):\n#    a.append(layer_module.shape)\n#---------intialize layer numbers and names dictionary------\nmodel_tupry.intialize_layer_name_num(model_tupry)\n\n#---------intialize pruned and remaning filters-----------\nmodel_tupry.filters_in_each_layer(model_tupry)\n \n#--------calculate remaining filters in each epoch--------\nmodel_tupry.remaining_filters_per_epoch(model=model_tupry,initial=True)\nmodel_tupry.calculate_total_flops(model_tupry)\nimport sys\nsys.exit()\nfor layer_name, layer_module in model_tupry.named_modules():\n           if(isinstance(layer_module, th.nn.Conv2d) or  isinstance(layer_module,th.nn.Linear)):\n              print(lay

partial_clustering

In [ ]:
import random

import torch
import torch as th
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
# from VGG16 import VGG16
import csv
from itertools import zip_longest
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from matplotlib.ticker import PercentFormatter
from torch.optim.lr_scheduler import MultiStepLR

seed =1787
random.seed(seed)
import os
os.environ['PYTHONHASHSEED'] = str(seed)
th.manual_seed(seed)
th.cuda.manual_seed(seed)
th.cuda.manual_seed_all(seed)
th.backends.cudnn.deterministic = True
#os.environ["CUDA_VISIBLE_DEVICES"]="1"




N = 1
batch_size_tr = 100
batch_size_te = 100
epochs = 2#300

threshold= 0.54
new_epochs=90#90-custom_epochs


th.cuda.set_device(0)
gpu = th.cuda.is_available()

if not gpu:
    print('qqqq')

else:

    transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = th.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = th.utils.data.DataLoader(testset, batch_size=100, shuffle=True, num_workers=2) 


total_layers=15
total_convs=13
total_blocks=total_convs

decision_count=th.ones((total_convs))

short=False
tr_size = 50000
te_size=10000


activation = 'relu'

#tr_size = 300
#te_size=300
#short=True

if gpu:
    model=VGG16(10,activation).cuda()
else:
    model=VGG16(10,activation)

folder_name=model.create_folders(total_convs)
logger=model.get_logger(folder_name+'logger.log')

optimizer = th.optim.SGD(model.parameters(), lr=0.1,momentum=0.9, weight_decay=5e-4)
scheduler = MultiStepLR(optimizer, milestones=[80,140,230], gamma=0.1)
criterion = nn.CrossEntropyLoss()

#ans=input("load from pretrained model= (t)true or (f)false")
ans='t'
if(ans=='t'):
  checkpoint = th.load('epoch_295.pth'
                       # ,map_location=torch.device('cpu')
                       )
  model.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  scheduler.load_state_dict(checkpoint['scheduler'])
  epoch_train_acc = checkpoint['train_acc']
  epoch_test_acc = checkpoint['test_acc']
  print('loading completed')

elif(ans=='f'):

    best_train_acc=0
    best_test_acc=0
    for n in range(N):

        #current_iteration = 0
        mi_iteration=0
        for epoch in range(epochs):
          #start=current_iteration
          train_acc=[]
          for batch_num, (inputs, targets) in enumerate(trainloader):
            if(batch_num==3 and short):
              break
            if(batch_num%100==0):
              print(batch_num)
            inputs = inputs.cuda()
            targets = targets.cuda()
            model.train()
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            with th.no_grad():
              y_hat = th.argmax(output, 1)
              score = th.eq(y_hat, targets).sum()
              train_acc.append(score.item())
              #current_iteration += 1
          
          with th.no_grad():
            #model.train_accuracy.append((sum(train_acc)*100)/tr_size) 
            epoch_train_acc=  (sum(train_acc)*100)/tr_size        
            test_acc=[]
            model.eval()
            for batch_nums, (inputs2, targets2) in enumerate(testloader):
                if(batch_nums==3 and short):
                    break

                inputs2, targets2 = inputs2.cuda(), targets2.cuda()            
                output=model(inputs2)
                y_hat = th.argmax(output, 1)
                score = th.eq(y_hat, targets2).sum()
                test_acc.append(score.item())

            #model.test_accuracy.append((sum(test_acc)*100)/te_size)
            epoch_test_acc= (sum(test_acc)*100)/te_size
            '''if(epoch_test_acc > best_test_acc ):
              best_test_acc=epoch_test_acc
              best_train_acc=epoch_train_acc '''      

          #end=current_iteration
          print('\n---------------Epoch number: {}'.format(epoch),
                  '---Train accuracy: {}'.format(epoch_train_acc),
                  '----Test accuracy: {}'.format(epoch_test_acc),'--------------')
          scheduler.step()
          print(optimizer.param_groups[0]['lr'])
else:
   print('wrong ans entered')
   import sys
   sys.exit()

#ended_epoch=epoch
#ended_iteration=end
prunes=0

#_____________________Conv_layers_________________
a=[]
for layer_name, layer_module in model.named_modules():
  if(isinstance(layer_module, th.nn.Conv2d)):
    a.append(layer_module)



d=[]
for i in range(total_convs):
      d.append(a[i].weight.shape[0])

d.append(epoch_train_acc)
d.append(epoch_test_acc)


with open(folder_name+'vggPrune.csv', 'a', newline='') as myfile:
          wr = csv.writer(myfile)
          command=model.get_writerow(total_convs+2)
          eval(command)
myfile.close()

ended_epoch=0
best_train_acc=epoch_train_acc
best_test_acc=epoch_test_acc

state = {'model': model.state_dict(),
          'train_acc': best_train_acc,
          'test_acc':best_test_acc,
          'optimizer':optimizer.state_dict(),
          'scheduler':scheduler.state_dict()}
#th.save(state,folder_name+'initial_pruning.pth')

decision=True
best_test_acc=0.0

'''with th.no_grad():
   final_first_ele =[]
   final_n_clusters=[]
   for layer_name, layer_module in model.named_modules():
          
          if(isinstance(layer_module, th.nn.Conv2d)):
 
                  #--------clustering to find indices-------
                  
                  clusters, ele = model.cluster_weights_agglo(layer_module.weight, threshold)

                  final_clusters.append(clusters)               
                  final_first_ele.append(ele)'''


while(decision==True):


    with th.no_grad():
      
      #_________________________PRUNING_EACH_CONV_LAYER__________________________     
 
      model.prune_filters(threshold,prunes,folder_name)
            

      #if(th.sum(decision_count)!=0):
      if(prunes==25):
          decision=False  
 
      prunes+=1

    d1=[]
    for i1 in range(total_convs):
      d1.append(a[i1].weight.shape[0])
    logger.info(d1)

    print('new-model starts....for ',new_epochs,' epochs')
    #print(model)

    optimizer = th.optim.SGD(model.parameters(), lr=0.01,momentum=0.9, weight_decay=5e-4)
    scheduler = MultiStepLR(optimizer, milestones=[40,70,90], gamma=0.1)
    #optimizer = th.optim.SGD(model.parameters(), lr=0.001,momentum=0.9, weight_decay=5e-4)
    #scheduler = MultiStepLR(optimizer, milestones=[40,70,90], gamma=0.1)
    
    best_test_acc=0.0

    for epoch in range(new_epochs):

          train_acc=[]
          test_acc=[]

          for batch_num, (inputs, targets) in enumerate(trainloader):

            if(batch_num==3 and short):
               break

            inputs = inputs.cuda()
            targets = targets.cuda()
            model= model.cuda()
            model.train()            
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            
            with th.no_grad():

               y_hat = th.argmax(output, 1)
               score = th.eq(y_hat, targets).sum()
               train_acc.append(score.item())
     

          with th.no_grad():            


              epoch_train_acc=(sum(train_acc)*100)/tr_size
              model.eval() 
          
              for batch_idx2, (inputs2, targets2) in enumerate(testloader):
                if(batch_idx2==3 and short):
                    break
                inputs2, targets2 = inputs2.cuda(), targets2.cuda()
                output=model(inputs2)
                y_hat = th.argmax(output, 1)
                score = th.eq(y_hat, targets2).sum()
                test_acc.append(score.item())

              epoch_test_acc=(sum(test_acc)*100)/te_size
              if(epoch_test_acc > best_test_acc ):
                      best_test_acc=epoch_test_acc
                      best_train_acc=epoch_train_acc
                      #state = {'model': model.state_dict(),
                      #         'train_acc': best_train_acc,
                      #         'test_acc':best_test_acc,
                      #         'optimizer':optimizer.state_dict(),
                      #         'scheduler':scheduler.state_dict()}
                      #th.save(state,folder_name+str(prunes)+'.pth')

          #print(optimizer.param_groups[0]['lr'])
          scheduler.step()
          
          logger.info('Epoch: {}/{}---Train:{:.3f}----Test: {:.3f}\n'.format(epoch,new_epochs-1,epoch_train_acc,epoch_test_acc))


    #----------------writing data-----------
    ended_epoch=ended_epoch+new_epochs

    d=[]
    for i in range(total_convs):
      d.append(a[i].weight.shape[0])
    d.append(best_train_acc)
    d.append(best_test_acc)
    with open(folder_name+'vggPrune.csv', 'a', newline='') as myfile:
          wr = csv.writer(myfile)
          command=model.get_writerow(total_convs+2)
          eval(command)

    myfile.close()
    #-------------------------end writing data---------------

from zipfile import ZipFile
import os,glob

directory = os.path.dirname(os.path.realpath(__file__)) #location of running file
file_paths = []
os.chdir(directory)
for filename in glob.glob("*.py"):
	filepath = os.path.join(directory, filename)
	file_paths.append(filepath)
	#print(filename)

print('Following files will be zipped:')
for file_name in file_paths:
	print(file_name)
saving_loc = folder_name #location of results
os.chdir(saving_loc)
# writing files to a zipfile
with ZipFile('python_files.zip','w') as zip:
	# writing each file one by one
	for file in file_paths:
		zip.write(file)


Files already downloaded and verified
Files already downloaded and verified
/content/drive/MyDrive
/
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv1
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv2
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv3
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv4
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv5
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv6
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv7
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv8
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv9
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv10
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv11
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv12
/content/drive/MyDrive/BTP/sem7\Results\Sep27_20_22_47PM_vgg/conv13
loading completed
0


09/27 08:27:04 PM | [44, 57, 125, 128, 256, 256, 256, 422, 352, 345, 370, 374, 375]
09/27 08:27:04 PM | [44, 57, 125, 128, 256, 256, 256, 422, 352, 345, 370, 374, 375]
INFO:gal:[44, 57, 125, 128, 256, 256, 256, 422, 352, 345, 370, 374, 375]


new-model starts....for  90  epochs


09/27 08:27:41 PM | Epoch: 0/89---Train:92.824----Test: 87.380

09/27 08:27:41 PM | Epoch: 0/89---Train:92.824----Test: 87.380

INFO:gal:Epoch: 0/89---Train:92.824----Test: 87.380

09/27 08:28:12 PM | Epoch: 1/89---Train:94.232----Test: 89.130

09/27 08:28:12 PM | Epoch: 1/89---Train:94.232----Test: 89.130

INFO:gal:Epoch: 1/89---Train:94.232----Test: 89.130

09/27 08:28:42 PM | Epoch: 2/89---Train:94.790----Test: 89.120

09/27 08:28:42 PM | Epoch: 2/89---Train:94.790----Test: 89.120

INFO:gal:Epoch: 2/89---Train:94.790----Test: 89.120

09/27 08:29:13 PM | Epoch: 3/89---Train:95.196----Test: 89.840

09/27 08:29:13 PM | Epoch: 3/89---Train:95.196----Test: 89.840

INFO:gal:Epoch: 3/89---Train:95.196----Test: 89.840

09/27 08:29:44 PM | Epoch: 4/89---Train:95.788----Test: 89.510

09/27 08:29:44 PM | Epoch: 4/89---Train:95.788----Test: 89.510

INFO:gal:Epoch: 4/89---Train:95.788----Test: 89.510

09/27 08:30:15 PM | Epoch: 5/89---Train:95.836----Test: 90.150

09/27 08:30:15 PM | Epoch: 5/89

1


...

09/27 10:06:39 PM | Epoch: 0/89---Train:96.068----Test: 86.550

09/27 10:06:39 PM | Epoch: 0/89---Train:96.068----Test: 86.550

INFO:gal:Epoch: 0/89---Train:96.068----Test: 86.550

09/27 10:07:06 PM | Epoch: 1/89---Train:96.248----Test: 87.570

09/27 10:07:06 PM | Epoch: 1/89---Train:96.248----Test: 87.570

INFO:gal:Epoch: 1/89---Train:96.248----Test: 87.570

09/27 10:07:32 PM | Epoch: 2/89---Train:96.510----Test: 89.500

09/27 10:07:32 PM | Epoch: 2/89---Train:96.510----Test: 89.500

INFO:gal:Epoch: 2/89---Train:96.510----Test: 89.500

09/27 10:07:59 PM | Epoch: 3/89---Train:96.510----Test: 90.060

09/27 10:07:59 PM | Epoch: 3/89---Train:96.510----Test: 90.060

INFO:gal:Epoch: 3/89---Train:96.510----Test: 90.060

09/27 10:08:26 PM | Epoch: 4/89---Train:97.030----Test: 90.550

09/27 10:08:26 PM | Epoch: 4/89---Train:97.030----Test: 90.550

INFO:gal:Epoch: 4/89---Train:97.030----Test: 90.550

09/27 10:08:54 PM | Epoch: 5/89---Train:96.890----Test: 90.260

09/27 10:08:54 PM | Epoch: 5/89

3


09/27 10:50:54 PM | [23, 53, 124, 128, 256, 256, 256, 364, 159, 103, 129, 135, 144]
09/27 10:50:54 PM | [23, 53, 124, 128, 256, 256, 256, 364, 159, 103, 129, 135, 144]
INFO:gal:[23, 53, 124, 128, 256, 256, 256, 364, 159, 103, 129, 135, 144]


new-model starts....for  90  epochs


09/27 10:51:20 PM | Epoch: 0/89---Train:96.336----Test: 89.410

09/27 10:51:20 PM | Epoch: 0/89---Train:96.336----Test: 89.410

INFO:gal:Epoch: 0/89---Train:96.336----Test: 89.410

09/27 10:51:47 PM | Epoch: 1/89---Train:96.322----Test: 89.590

09/27 10:51:47 PM | Epoch: 1/89---Train:96.322----Test: 89.590

INFO:gal:Epoch: 1/89---Train:96.322----Test: 89.590

09/27 10:52:12 PM | Epoch: 2/89---Train:96.602----Test: 89.500

09/27 10:52:12 PM | Epoch: 2/89---Train:96.602----Test: 89.500

INFO:gal:Epoch: 2/89---Train:96.602----Test: 89.500

09/27 10:52:38 PM | Epoch: 3/89---Train:96.606----Test: 89.360

09/27 10:52:38 PM | Epoch: 3/89---Train:96.606----Test: 89.360

INFO:gal:Epoch: 3/89---Train:96.606----Test: 89.360



In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv1" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv2" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv3" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv4" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv5" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv6" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv7" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv8" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv9" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv10" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv11" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv12" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# !cp -r "/\Results\Sep27_18_18_02PM_vggconv13" "/content/drive/MyDrive/btpsem7data"

In [ ]:
# 